### PROBLEM STATEMENT

Write a script that ingests JSON data and transforms into a data table that is easy to use for further analysis.

Considerations:

1. The column packageName can be found from the JSON key “packageName”
2. The column sku can be found from the JSON key “sku”
3. The column countryCode can be found from the JSON keys that contain two-letter country codes nested    within the JSON key “prices”
4. The column currency can be found from the JSON key “currency”
5. The column price can be found from the JSON key “priceMicros”. The price is equal to
   “priceMicros” divided by 1,000,000.

Required data columns after transformation: packageName, sku, countryCode, currency, price


In [2]:
import json 
import pandas as pd

# Loading the given sample  data
with open('C:\Users\Madhu\Side Projects\Case Studies\sample_json.txt') as txt_data:
     json_data = json.load(txt_data)

### APPROACHES

1. If a price or currency value is missing, ignore the missing data point and only ouptut the data with both price and currency values

2. If a price or currency value is missing, replace it with the default price and currency. (This can also be modified to replace with NA depending on requirements)

To solve the purpose, a function "json_to_df" is created. "json_to_df" takes two arguments, "json", which is the json input data and "ignore_missing" which is a boolean value : "True" -Ignore records which has missing values, "False"- Replace missing values with default values. 

The sample data set provided does not have any missing data for the keys, price and currency. Hence running the below code, outputs the required table with all data points from the sample text file irrespective of the boolean value passed. 



In [27]:
def json_to_df(json, ignore_missing):
    data = []
    if ignore_missing == True:
    # using list comprehension technique to add the required key value pairs for the final output table        
        data =  [   
            { "packageName" : keys["packageName"],
               "sku" : keys["sku"],
               "countryCode" : key,
               "currency" : value["currency"],
               "price" : float(value["priceMicros"])/1000000
             }
    
                for keys in json["inappproduct"]
                for key, value in keys["prices"].items()
                if isinstance(value, dict) and "currency" in value and "priceMicros" in value
         ]
    
    elif ignore_missing == False:
    # using loops to perform the similar previous step to append required key value pairs to the list        
        for keys in json["inappproduct"]:
            for key, value in keys["prices"].items():
                if isinstance (value,dict) and "currency" in value and "priceMicros" in value:
                    data.append(
                        {"packageName" : keys["packageName"],
                          "sku" : keys["sku"],
                          "countryCode" : key,
                          "currency" : value["currency"],
                          "price" : float(value["priceMicros"])/1000000
                         }
                     )
                else:
                    if isinstance (value,dict) and ("currency" not in value or "priceMicros" not in value) and len(key)<2:
                        data.append(
                            {"packageName" : keys["packageName"],
                             "sku" : keys["sku"],
                             "countryCode" : key,
                             "currency" : keys["defaultPrice"]["currency"],
                             "price" : float(keys["defaultPrice"]["priceMicros"])/1000000
                            }
                        )            
    # Converting the list to a DataFrame for exporting it to a table 
    dataPoints = pd.DataFrame(data).reindex(columns = ["packageName", 
                                                       "sku", 
                                                       "countryCode",
                                                       "currency", 
                                                       "price"]
                                           )

    return dataPoints
    
json_to_df(json_data,True)    

,packageName,sku,countryCode,currency,price
0,gamehive_game1,gamehive_game1_product1,RU,RUB,3790.00
1,gamehive_game1,gamehive_game1_product1,CA,CAD,69.99
2,gamehive_game1,gamehive_game1_product1,JP,JPY,6000.00
3,gamehive_game1,gamehive_game1_product1,IT,EUR,54.99
4,gamehive_game1,gamehive_game1_product1,US,USD,49.99
5,gamehive_game1,gamehive_game1_product1,KR,KRW,65000.00
6,gamehive_game1,gamehive_game1_product1,AU,AUD,79.99
7,gamehive_game1,gamehive_game1_product1,GB,GBP,48.99
8,gamehive_game1,gamehive_game1_product1,DZ,DZD,6325.00
9,gamehive_game1,gamehive_game1_product1,SG,SGD,68.98



To demonstrate the working of this function with missing values, the sample data was modified to remove currency of the key DZ and PriceMicros of the key JP under the sku - gamehive_game1_product1



In [28]:
#loading the modified data onto a test json object

with open('C:\Users\Madhu\Side Projects\Case Studies\Test_cases.txt') as test_data:
    json_test_data = json.load(test_data)

     

In [29]:
   
json_to_df(json_test_data,True)  

,packageName,sku,countryCode,currency,price
0,gamehive_game1,gamehive_game1_product1,RU,RUB,3790.00
1,gamehive_game1,gamehive_game1_product1,CA,CAD,69.99
2,gamehive_game1,gamehive_game1_product1,IT,EUR,54.99
3,gamehive_game1,gamehive_game1_product1,US,USD,49.99
4,gamehive_game1,gamehive_game1_product1,KR,KRW,65000.00
5,gamehive_game1,gamehive_game1_product1,AU,AUD,79.99
6,gamehive_game1,gamehive_game1_product1,GB,GBP,48.99
7,gamehive_game1,gamehive_game1_product1,SG,SGD,68.98
8,gamehive_game1,gamehive_game1_bundle1,RU,RUB,7490.00
9,gamehive_game1,gamehive_game1_bundle1,CA,CAD,139.99


As seen from the above output, the missing value for countryCode JP and countryCode DZ has been ignored and only 17 data rows with non missing price and currency has been fetched in the output


In [31]:
   
json_to_df(json_test_data,False)  

,packageName,sku,countryCode,currency,price
0,gamehive_game1,gamehive_game1_product1,RU,RUB,3790.00
1,gamehive_game1,gamehive_game1_product1,CA,CAD,69.99
2,gamehive_game1,gamehive_game1_product1,IT,EUR,54.99
3,gamehive_game1,gamehive_game1_product1,US,USD,49.99
4,gamehive_game1,gamehive_game1_product1,KR,KRW,65000.00
5,gamehive_game1,gamehive_game1_product1,AU,AUD,79.99
6,gamehive_game1,gamehive_game1_product1,GB,GBP,48.99
7,gamehive_game1,gamehive_game1_product1,SG,SGD,68.98
8,gamehive_game1,gamehive_game1_bundle1,RU,RUB,7490.00
9,gamehive_game1,gamehive_game1_bundle1,CA,CAD,139.99


As seen from the above output, index 2 - with missing value for countryCode JP has been replaced with default value in CAD and index 8 - with missing value for countryCode DZ has been replaced wirh default value in CAD